In [1]:
from qiskit import QuantumCircuit
import numpy as np

from qbench.schemes.postselection import benchmark_discrimination_using_postselection
from qbench.schemes.direct_sum import benchmark_discrimination_using_direct_sum

In [2]:
from qiskit_aer import StatevectorSimulator

simulator = StatevectorSimulator()

In [4]:
### discrimination experiment for measurement in Hadamard basis using postselection and direct sum###

def state_prep():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit.to_instruction()

def u_dag():
    circuit = QuantumCircuit(1)
    circuit.h(0)
    return circuit.to_instruction()

def v0_dag():
    circuit = QuantumCircuit(1)
    circuit.ry(-np.pi * 3 / 4, 0)
    return circuit.to_instruction()
    
def v1_dag():
    circuit = QuantumCircuit(1)
    circuit.ry(-np.pi * 3 / 4, 0)
    circuit.x(0)
    return circuit.to_instruction()
    
def v0_v1_direct_sum_dag():
    circuit = QuantumCircuit(2)
    circuit.ry(-np.pi * 3 / 4, 0)
    circuit.cx(0, 1)
    return circuit.to_instruction()

In [5]:
discrimination_postselection_result = benchmark_discrimination_using_postselection(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_dag=v0_dag(),
    v1_dag=v1_dag(),
    num_shots_per_measurement=100000,)

In [6]:
discrimination_direct_sum_result = benchmark_discrimination_using_direct_sum(
    backend=simulator,
    target=1,
    ancilla=2,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_v1_direct_sum_dag=v0_v1_direct_sum_dag(),
    num_shots_per_measurement=100000,)

In [7]:
p_succ = (2 + np.sqrt(2)) / 4
print(f"Analytical p_succ = {p_succ}")
print(f"Postselection: p_succ = {discrimination_postselection_result}, abs.error ={np.abs(p_succ - discrimination_postselection_result)}")
print(f"Direct sum: p_succ = {discrimination_direct_sum_result}, abs.error ={np.abs(p_succ - discrimination_direct_sum_result)}")

In [8]:
from qbench.schemes.postselection import (
    assemble_circuits_discrimination_postselection,
    compute_probabilities_discrimination_postselection,)

circuits = assemble_circuits_discrimination_postselection(
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_dag=v0_dag(),
    v1_dag=v1_dag(),)

In [9]:
from qiskit_aer.noise import NoiseModel, ReadoutError

error = ReadoutError([[0.75, 0.25], [0.8, 0.2]])
noise_model = NoiseModel()
noise_model.add_readout_error(error, [0])
noise_model.add_readout_error(error, [1])


In [10]:
keys_ordering = ["id_v0", "id_v1", "u_v0", "u_v1"]

all_circuits = [circuits[key] for key in keys_ordering]

counts_noisy = simulator.run(
    all_circuits,
    backend=simulator,
    noise_model=noise_model,
    shots=10000).result().get_counts()

counts_noiseless = simulator.run(
    all_circuits,
    backend=simulator,
    shots=10000).result().get_counts()

In [11]:
prob_succ_noiseless = compute_probabilities_discrimination_postselection(
    id_v0_counts=counts_noiseless[0],
    id_v1_counts=counts_noiseless[1],
    u_v0_counts=counts_noiseless[2],
    u_v1_counts=counts_noiseless[3],)

prob_succ_noisy = compute_probabilities_discrimination_postselection(
    id_v0_counts=counts_noisy[0],
    id_v1_counts=counts_noisy[1],
    u_v0_counts=counts_noisy[2],
    u_v1_counts=counts_noisy[3],)

In [12]:
p_succ = (2 + np.sqrt(2)) / 4
print(f"Analytical p_succ = {p_succ}")
print(f"Prob_succ_noiseless {prob_succ_noiseless}, abs.error ={np.abs(p_succ - prob_succ_noiseless)}")
print(f"Prob_succ_noisy = {prob_succ_noisy}, abs.error ={np.abs(p_succ - prob_succ_noisy)}")